In [ ]:
! git clone https://github.com/predictive-clinical-neuroscience/PCNtoolkit-demo.git

In [ ]:
import os

In [ ]:
os.chdir('/content/PCNtoolkit-demo/

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats, linalg
from sklearn import preprocessing, decomposition, linear_model, metrics 
import warnings

In [ ]:
# set fontsizes for matplotlib plots
baseline_fontsize = 12
SMALL_SIZE = 8 + baseline_fontsize
MEDIUM_SIZE = 10 + baseline_fontsize
BIGGER_SIZE = 12 + baseline_fontsize

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Load Data

In [ ]:
all_subs = np.load('data/all_subs_regression_data.npy')
gscores = np.load('data/g_labels.npy')

In [ ]:
print(all_subs.shape)
print(gscores.shape)

# Create Train/Test Splits

In [ ]:
# generate train/test splits
np.random.seed(42)
n_train = int(0.8 * all_subs.shape[0])

train_idxs = np.random.choice(range(all_subs_T1.shape[0]), size=n_train, replace=False)
test_idxs = np.array([x for x in range(all_subs.shape[0]) if x not in train_idxs])

In [ ]:
train_data = all_subs[train_idxs, :]
test_data = all_subs[test_idxs, :]

train_phen = gscores[train_idxs]
test_phen = gscores[test_idxs]

In [ ]:
# mean center train/test data (using train means)
train_mu_centered = (train_data - train_data.mean(axis=0))
test_mu_centered = (test_data - train_data.mean(axis=0))

# Principal Component Regression (BBS)

In [ ]:
pca_model = decomposition.PCA(n_components=75).fit(train_data)
# from pca documentation, "the input data is centered but not scaled for each feature before applying the SVD"

In [ ]:
print(f'First PC explains {pca_model.explained_variance_ratio_[0]*100:.2f}% of the total variance.\nThis is an artifact of zero inflated data')
plt.figure(figsize=(10, 7))
plt.bar(range(1, 51), pca_model.explained_variance_ratio_[1:51])
plt.title('Variance Explained Ratio\nPCs 1-50', fontsize=25)
plt.show()

In [ ]:
train_ransformed = pca_model.transform(train_data)
test_transformed = pca_model.transform(test_data)

## Fit Linear Regression Model 

In [ ]:
# fast OLS using matrix math
# we will check that this matches sklearn results later

# fit ols model on dimension reduced train data
train_features = np.hstack([np.ones((train_transformed.shape[0], 1)), 
                            train_transformed])
train_features_inv = linalg.pinv2(train_features)
train_betas = np.dot(train_features_inv, train_phen)
train_pred_phen = np.dot(train_features, train_betas)

# fit ols model on dimension reduced test data
test_features = np.hstack([np.ones((test_transformed.shape[0], 1)), 
                           test_transformed])
test_pred_phen = np.dot(test_features, train_betas)

In [ ]:
# OLS using sklearn

lr_model = linear_model.LinearRegression(fit_intercept=True, normalize=False)
lr_model.fit(train_transformed, train_phen)
train_pred_phen_lr_model = lr_model.predict(train_transformed)
test_pred_phen_lr_model = lr_model.predict(test_transformed)

In [ ]:
# ensure matrix math predictions and sklearn predictions are accurate to 5 decimals
assert np.allclose(np.round(train_pred_phen - train_pred_phen_lr_model, 5), 0), 'Failed'
assert np.allclose(np.round(test_pred_phen - test_pred_phen_lr_model, 5), 0), 'Failed'
print('Passed')

## Accuracy of Predictions

In [ ]:
train_r2 = metrics.r2_score(train_phen, train_pred_phen_lr_model)
train_mae = metrics.mean_absolute_error(train_phen, train_pred_phen_lr_model)
test_mae = metrics.mean_absolute_error(test_phen, test_pred_phen_lr_model)
train_mae = metrics.mean_squared_error(train_phen, train_pred_phen_lr_model)
test_mae = metrics.mean_squared_error(test_phen, test_pred_phen_lr_model)
print(f'Train R^2: {train_r2:.3f}')
print(f'Train MAE: {train_mae:.3f}')
print(f'Test MAE: {test_mae:.3f}')
print(f'Train MSE: {train_mae:.3f}')
print(f'Test MSE: {test_mae:.3f}')

## BBS Cross Validation

In [ ]:
def bbs(X, y, n_components, n_cv_splits, pred_summary_function, verbose=False):
    assert X.shape[0] == y.shape[0]
    
    fold_accs_train = []
    fold_accs_test = []
    np.random.seed(42)
    shuffled_idxs = np.random.choice(range(X.shape[0]), size=X.shape[0], replace=False)
    for fold_i, test_idxs in enumerate(np.array_split(shuffled_idxs, n_cv_splits)):
        train_mask = np.ones(X.shape[0], np.bool)
        train_mask[test_idxs] = 0

        # create train/text X, y
        train_X, test_X = X[train_mask, :], X[test_idxs, :]
        train_y, test_y = y[train_mask], y[test_idxs]  

        # mean center columns using train data only
        train_X_mu = train_X.mean(axis=0)
        train_X = train_X - train_X_mu
        test_X = test_X - train_X_mu

        # fit pca
        if verbose:
            print(f'CV Fold: {fold_i+1:<10} Fitting PCA model...')
        pca_model = decomposition.PCA(n_components=n_components).fit(train_X)

        # dimension reduce train/test data
        train_X = pca_model.transform(train_X)
        test_X = pca_model.transform(test_X)

        # fit OLS model
        if verbose:
            print(f'CV Fold: {fold_i+1:<10} Fitting Linear Regression model...')
        lr_model = linear_model.LinearRegression(fit_intercept=True, normalize=False)
        lr_model.fit(train_X, train_y)

        train_pred = lr_model.predict(train_X)
        test_pred = lr_model.predict(test_X)

        fold_accs_train.append(pred_summary_function(train_y, train_pred))
        fold_accs_test.append(pred_summary_function(test_y, test_pred))
        
        if verbose:
            print(f'CV Fold: {fold_i+1:<10} Train Accuracy: {round(fold_accs_train[-1], 3):<10} Test Accuracy: {round(fold_accs_test[-1], 3):<10}')

    
    plt.figure(figsize=(13, 7))
    plt.plot(range(1, len(fold_accs_train)+1), fold_accs_train, linestyle='-', marker='o', color='C0', label='Train CV Performance')
    plt.plot(range(1, len(fold_accs_test)+1), fold_accs_test, linestyle='-', marker='o', color='C1', label='Test CV Performance')
    plt.title(pred_summary_function.__name__, fontsize=20)
    plt.xticks(range(1, len(fold_accs_test)+1))
    plt.xlabel('CV Fold')
    plt.legend(fontsize=20)
    plt.show()
    
    return fold_accs_train, fold_accs_test

In [ ]:
fold_accs_train, fold_accs_test = bbs(all_subs, gscores, n_components=75, n_cv_splits=5, pred_summary_function=metrics.mean_absolute_error, verbose=True)

# Connectome Predictive Modelling 

In [ ]:
# correlation train_brain with train_phenotype
train_pheno_corr_p = [stats.pearsonr(train_data[:, i], train_phen) for i in range(train_data.shape[1])]  # train_pheno_corr_p: (259200, )
# there are some nan correlations if brain data is poorly cropped (ie: some columns are always 0)

In [ ]:
# split into positive and negative correlations 
# and keep edges with p values below threshold
pval_threshold = 0.01

train_corrs = np.array([x[0] for x in train_pheno_corr_p])
train_pvals = np.array([x[1] for x in train_pheno_corr_p])

keep_edges_pos = (train_corrs > 0) & (train_pvals < pval_threshold)
keep_edges_neg = (train_corrs < 0) & (train_pvals < pval_threshold)

In [ ]:
print(f'number of positive features kept = {np.sum(keep_edges_pos)}')
print(f'number of negative features kept = {np.sum(keep_edges_neg)}')

In [ ]:
train_pos_edges_sum = train_T1[:, keep_edges_pos].sum(1)
train_neg_edges_sum = train_T1[:, keep_edges_neg].sum(1)

In [ ]:
fit_pos = linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(train_pos_edges_sum.reshape(-1, 1), train_phen)
fit_neg = linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(train_neg_edges_sum.reshape(-1, 1), train_phen)

In [ ]:
pos_error = metrics.mean_absolute_error(train_phen, fit_pos.predict(train_pos_edges_sum.reshape(-1, 1)))
neg_error = metrics.mean_absolute_error(train_phen, fit_neg.predict(train_neg_edges_sum.reshape(-1, 1)))

print(f'Training Error (Positive Features Model) = {pos_error:.3f}')
print(f'Training Error (Negative Features Model) = {neg_error:.3f}')

In [ ]:
# combine positive/negative edges in one linear regression model
fit_pos_neg = linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(np.stack((train_pos_edges_sum, train_neg_edges_sum)).T, train_phen)

In [ ]:
pos_neg_error = metrics.mean_absolute_error(train_phen, fit_pos_neg.predict(np.stack((train_pos_edges_sum, train_neg_edges_sum)).T))

print(f'Training Error (Positive/Negative Features Model) = {pos_neg_error:.3f}')

In [ ]:
# evaluate out of sample performance 
test_pos_edges_sum = test_data[:, keep_edges_pos].sum(1)
test_neg_edges_sum = test_data[:, keep_edges_neg].sum(1)

pos_test_error = metrics.mean_absolute_error(test_phen, fit_pos.predict(test_pos_edges_sum.reshape(-1, 1)))
neg_test_error = metrics.mean_absolute_error(test_phen, fit_neg.predict(test_neg_edges_sum.reshape(-1, 1)))
pos_neg_test_error = metrics.mean_absolute_error(test_phen, fit_pos_neg.predict(np.stack((test_pos_edges_sum, test_neg_edges_sum)).T))

print(f'Testing Error (Positive Features Model) = {pos_test_error:.3f}')
print(f'Testing Error (Negative Features Model) = {neg_test_error:.3f}')
print(f'Testing Error (Positive/Negative Features Model) = {pos_neg_test_error:.3f}')

## CPM Cross Validation

In [ ]:
def cpm(X, y, p_threshold, n_cv_splits, pred_summary_function, verbose=False):
    assert X.shape[0] == y.shape[0]
    
    fold_accs_train = []
    fold_accs_test = []
    np.random.seed(42)
    shuffled_idxs = np.random.choice(range(X.shape[0]), size=X.shape[0], replace=False)
    for fold_i, test_idxs in enumerate(np.array_split(shuffled_idxs, n_cv_splits)):
        train_mask = np.ones(X.shape[0], np.bool)
        train_mask[test_idxs] = 0

        # create train/text X, y
        train_X, test_X = X[train_mask, :], X[test_idxs, :]
        train_y, test_y = y[train_mask], y[test_idxs]  
        
        # create correlation matrix between train_X and train_y
        if verbose:
            print(f'CV Fold: {fold_i+1:<10} Computing correlations between train_X and train_y...')
        with warnings.catch_warnings():
            # we expect pearsonr to throw PearsonRConstantInputWarning because of contant valued columns in X
            warnings.simplefilter("ignore")
            train_pheno_corr_p = [stats.pearsonr(train_X[:, i], train_y) for i in range(train_X.shape[1])]
            train_corrs = np.array([x[0] for x in train_pheno_corr_p])
            train_pvals = np.array([x[1] for x in train_pheno_corr_p])
            # create masks for edges below p-threshold and split pos/neg correlations
            keep_edges_pos = (train_corrs > 0) & (train_pvals < p_threshold)
            keep_edges_neg = (train_corrs < 0) & (train_pvals < p_threshold)
        
        # sum X entries with significant correlations with y
        train_pos_edges_sum = train_X[:, keep_edges_pos].sum(1)
        train_neg_edges_sum = train_X[:, keep_edges_neg].sum(1)
        test_pos_edges_sum = test_X[:, keep_edges_pos].sum(1)
        test_neg_edges_sum = test_X[:, keep_edges_neg].sum(1)
        
        # fit linear regression models based on summed values
        fit_pos = linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(train_pos_edges_sum.reshape(-1, 1), train_y)
        fit_neg = linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(train_neg_edges_sum.reshape(-1, 1), train_y)
        fit_pos_neg = linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(np.stack((train_pos_edges_sum, train_neg_edges_sum)).T, train_y)
        
        # compute train errors
        train_pos_error = pred_summary_function(train_y, fit_pos.predict(train_pos_edges_sum.reshape(-1, 1)))
        train_neg_error = pred_summary_function(train_y, fit_neg.predict(train_neg_edges_sum.reshape(-1, 1)))
        train_posneg_error = pred_summary_function(train_y, fit_pos_neg.predict(np.stack((train_pos_edges_sum, train_neg_edges_sum)).T))

        # compute testing errors
        test_pos_error = pred_summary_function(test_y, fit_pos.predict(test_pos_edges_sum.reshape(-1, 1)))
        test_neg_error = pred_summary_function(test_y, fit_neg.predict(test_neg_edges_sum.reshape(-1, 1)))
        test_posneg_error = pred_summary_function(test_y, fit_pos_neg.predict(np.stack((test_pos_edges_sum, test_neg_edges_sum)).T))

        fold_accs_train.append((train_pos_error, train_neg_error, train_posneg_error))
        fold_accs_test.append((test_pos_error, test_neg_error, test_posneg_error))
        
        if verbose:
            print(f'CV Fold: {fold_i+1:<10} Train Pos-Edges Model Accuracy: {round(train_pos_error, 3):<10} Train Neg-Edges Model Accuracy: {round(train_neg_error, 3):<10} Train Pos/Neg-Edges Model Accuracy: {round(train_posneg_error, 3):<10}')
            print(f'CV Fold: {fold_i+1:<10} Test  Pos-Edges Model Accuracy: {round(test_pos_error, 3):<10} Test  Neg-Edges Model Accuracy: {round(test_neg_error, 3):<10} Test  Pos/Neg-Edges Model Accuracy: {round(test_posneg_error, 3):<10}')

    
    plt.figure(figsize=(13, 7))
    plt.plot(range(1, len(fold_accs_train)+1), [x[0] for x in fold_accs_train], linestyle='--', marker='o', color='C0', label='Train Pos-Edges Model')
    plt.plot(range(1, len(fold_accs_train)+1), [x[1] for x in fold_accs_train], linestyle='--', marker='o', color='C1', label='Train Neg-Edges Model')
    plt.plot(range(1, len(fold_accs_train)+1), [x[2] for x in fold_accs_train], linestyle='--', marker='o', color='C2', label='Train Pos/Neg-Edges Model')
    
    plt.plot(range(1, len(fold_accs_test)+1), [x[0] for x in fold_accs_test], linestyle='-', marker='o', color='C0', label='Test  Pos-Edges Model')
    plt.plot(range(1, len(fold_accs_test)+1), [x[1] for x in fold_accs_test], linestyle='-', marker='o', color='C1', label='Test  Neg-Edges Model')
    plt.plot(range(1, len(fold_accs_test)+1), [x[2] for x in fold_accs_test], linestyle='-', marker='o', color='C2', label='Test  Pos/Neg-Edges Model')
    
    plt.title(pred_summary_function.__name__, fontsize=20)
    plt.xticks(range(1, len(fold_accs_test)+1))
    plt.xlabel('CV Fold')
    plt.legend(fontsize=20)
    plt.show()
    
    return fold_accs_train, fold_accs_test

In [ ]:
fold_accs_train, fold_accs_test = cpm(all_subs, gscores, p_threshold=0.01, n_cv_splits=5, pred_summary_function=metrics.mean_absolute_error, verbose=True)

# Lasso (Linear Regression + L1 Regularization)

In [ ]:
# LassoCV uses coordinate descent to select hyperparameter alpha 
alpha_grid = np.array([10**a for a in np.arange(-3, 3, 0.25)])
lassoCV_model = linear_model.LassoCV(cv=5, n_alphas=len(alpha_grid), alphas=alpha_grid, fit_intercept=True, normalize=False, random_state=42, verbose=True, n_jobs=5).fit(train_data, train_phen)

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(lassoCV_model.alphas_, lassoCV_model.mse_path_, ':')
plt.plot(lassoCV_model.alphas_, lassoCV_model.mse_path_.mean(axis=-1), color='k', marker='o', label='Mean MSE Across Folds', linewidth=2)
plt.axvline(x=100, linestyle='--', c='r')
plt.xlabel('Alpha')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
# based on cv results above, set alpha=100
lasso_model = linear_model.Lasso(alpha=lassoCV_model.alpha_, fit_intercept=True, normalize=False).fit(train_data, train_phen)

In [ ]:
train_preds_lasso_model = lasso_model.predict(train_data)
test_preds_lasso_model = lasso_model.predict(test_data)

train_mae = metrics.mean_absolute_error(train_phen, train_preds_lasso_model)
test_mae = metrics.mean_absolute_error(test_phen, test_preds_lasso_model)

print(f'Train MAE: {train_mae:.3f}')
print(f'Test MAE: {test_mae:.3f}')

# Ridge (Linear Regression + L2 Regularization)

In [ ]:
# RidgeCV uses generalized cross validation to select hyperparameter alpha 
with warnings.catch_warnings():
    # ignore matrix decomposition errors
    warnings.simplefilter("ignore")
    ridgeCV_model = linear_model.RidgeCV(alphas=(0.1, 1.0, 10.0), fit_intercept=True, normalize=False, cv=5).fit(train_data, train_phen)

In [ ]:
ridge_alpha = ridgeCV_model.alpha_
print(f'CV Selected Alpha = {ridge_alpha:.3f}')

In [ ]:
ridge_model = linear_model.Ridge(alpha=ridge_alpha, fit_intercept=True, normalize=False).fit(train_data, train_phen)

In [ ]:
train_preds_ridge_model = ridge_model.predict(train_data)
test_preds_ridge_model = ridge_model.predict(test_data)

train_mae = metrics.mean_absolute_error(train_phen, train_preds_ridge_model)
test_mae = metrics.mean_absolute_error(test_phen, test_preds_ridge_model)

print(f'Train MAE: {train_mae:.3f}')
print(f'Test MAE: {test_mae:.3f}')

# Elastic Net (Linear Regression + L1/L2 Regularization)

In [ ]:
# RidgeCV uses generalized cross validation to select hyperparameter alpha 
elasticnetCV_model = linear_model.ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=5, n_alphas=len(alpha_grid), alphas=alpha_grid, random_state=42, verbose=True, n_jobs=5).fit(train_data, train_phen)

In [ ]:
print(f'CV selected alpha {elasticnetCV_model.alpha_:.3f}')
print(f'Elastic net L1 ratio {elasticnetCV_model.l1_ratio_:.3f}')

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(elasticnetCV_model.alphas_, elasticnetCV_model.mse_path_[1, :, :], ':')
plt.plot(elasticnetCV_model.alphas_, elasticnetCV_model.mse_path_[1, :, :].mean(axis=-1), color='k', marker='o', label='Mean MSE Across Folds', linewidth=2)
plt.axvline(x=200, linestyle='--', c='r')
plt.xlabel('Alpha')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
elasticnet_model = linear_model.ElasticNet(alpha=elasticnetCV_model.alpha_, l1_ratio=elasticnetCV_model.l1_ratio_, fit_intercept=True, normalize=False, random_state=42).fit(train_data, train_phen)

train_preds_en_model = elasticnet_model.predict(train_data)
test_preds_en_model = elasticnet_model.predict(test_data)

train_mae = metrics.mean_absolute_error(train_phen, train_preds_en_model)
test_mae = metrics.mean_absolute_error(test_phen, test_preds_en_model)

print(f'Train MAE: {train_mae:.3f}')
print(f'Test MAE: {test_mae:.3f}')